## 제주도 관광 이슈 크롤링
네이버 뉴스에서 해당하는 달의 관광 이슈 크롤링

In [1]:
from ssl import SSLError
from urllib import parse
from urllib.error import URLError
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import time
import socket
import random
import argparse
import datetime
import pandas as pd

In [56]:
def crawl(query, save_as, begin, end, sort=0, field=1, delay=0.5, timeout=30, page_limit=50):
    '''
    :param query: 네이버 '뉴스'란에서 검색할 검색어
    :param save_as: 검색 결과 저장 경로
    :param begin: '기간' -> 검색 기간 시작
    :param end: '기간' -> 검색 기간 끝
    :param sort: '유형' -> 0(관련도순) 1(최신순) 2(오래된순)
    :param field: '영역' -> 0(전체) 1(제목)
    :param delay: (옵션) 검색 리퀘스트 간격 (초)
    :param timeout: (옵션) 타임아웃 시 기다릴 시간 (초)
    :param page_limit: (옵션) 검색 결과에서 몇 페이지까지 갈 것인지 결정
    :return:
    '''

    # prerequisite
    df = pd.DataFrame(columns=['link', 'title', 'date', 'article'])

    # index settings
    # a single pages includes 10 news, starting from page 1 (index 1~10)
    current_index = 1
    max_index = 2

    while (current_index <= max_index) and (1 + current_index // 10 <= page_limit):
        print('\n' + 'crawling... %s (current_page / max_page %i/%i)' % (query, 1 + current_index // 10, 1 + max_index // 10))
        url = make_url(query, sort, field, begin, end, current_index)
        print('making url', url)

        print('making beautifulsoup object from html')
        bsobj = make_bsobj(url, delay, timeout, trial=10)

        if bsobj is None:
            continue
        print('extracting naver news urls from bsobj')
        naver_news_urls = make_naver_news_urls(bsobj)
        naver_news_title = get_naver_news_title(bsobj)
        print(naver_news_urls)
        print(naver_news_title)

        for url in naver_news_urls:
            print('\topening:', url)
            news_bsobj = BeautifulSoup(url, 'html.parser')
            
            naver_news_article = get_naver_news_title(news_bsobj)

            if news_bsobj is None:
                continue

            attributes = get_attributes(news_bsobj)
            if attributes is None:
                continue
            naver_news_date = get_naver_news_date(news_bsobj)

            date, article, title, newspaper = attributes
            df = df.append({'link': url,
                            'title': title,
                            'newspaper': newspaper,
                            'date': date,
                            'article': article}, ignore_index=True)
            print('\t', title)

        print('saving updated df')
        df = df.sort_values(by=['date'])
        df.to_excel(save_as, engine='xlsxwriter')

        print('updating current_news_index info')
        current_index += 10
        max_index = get_max_index(bsobj)
        if max_index is None:
            break

In [48]:
def make_url(query, sort, field, begin, end, page):
    url = "https://search.naver.com/search.naver?&where=news&query=" + parse.quote(query)
    url += "&sort=%i" % sort
    url += "&field=%i" % field
    url += "&ds=" + begin + "&de=" + end
    url += "&nso=so:r,p:"
    url += "from" + begin.replace(".", "") + "to" + end.replace(".", "")
    url += "&start=" + str(page)
    url += "&refresh_start=0"
    return url

In [49]:
def make_bsobj(url, delay=0.5, timeout=30, trial=10):
    ua = UserAgent(verify_ssl=False)
    count = 0

    while count < trial:
        try:
            time.sleep(delay + random.random())
            html = urlopen(Request(url=url, headers={'User-Agent': ua.random}), timeout=timeout)
            bsobj = BeautifulSoup(html, 'html.parser')
            return bsobj
        except (URLError, SSLError, socket.timeout) as e:
            print('(Error)', e)
            print('reloading...')
            count += 1
            time.sleep(timeout)
    return None

In [55]:
def make_naver_news_urls(bsobj):
    return [link['href'] for link in bsobj.find_all('a', 'news_tit')]

def get_naver_news_title(bsobj):
    return [link.text for link in bsobj.find_all('a', 'news_tit')]

def get_naver_news_article(news_bsobj):
    return [link.text for link in news_bsobj.find_all()]

In [51]:
def get_attributes(bsobj):
    def _get_title(bsobj):
        ls = []
        title = bsobj.find('a', 'news_tit').text
        ls.append(title)
        print(ls)
        return title

    def _get_article(bsobj):
        article = bsobj.select('#articleBodyContents')[0].text
        article = article.encode('utf-8', 'replace').decode()
        return article

    def _get_date(news_bsobj):
        splits = bsobj.select('.t11')[0].text.split(' ')
        date = splits[0] + ' ' + splits[2]
        date = datetime.datetime.strptime(date, '%Y.%m.%d. %H:%M')
        date += datetime.timedelta(hours=12 * int(splits[1] == '오후'))
        print(date)
        return date

    def _get_newspaper(bsobj):
        newspaper = bsobj.find("div", class_="press_logo").find('img', alt=True).get('alt')
        return newspaper

    try:
        return _get_date(bsobj), _get_article(bsobj), _get_title(bsobj), _get_newspaper(bsobj)
    except IndexError:
        print('(Error) crawling failed (maybe url is redirected to somewhere else)')
        return None

In [52]:
def get_max_index(bsobj):
    paging = bsobj.find("div", {"class": "sc_page_inner"})
    if not paging:
        print('(WARNING!) no results found')
        return None

    atags = paging.find_all('a')
    if not atags:
        print('(WARNING!) there is only one page')
        return None

    return max([int(atag["href"].split('start=')[1]) for atag in atags])

In [53]:
def get_arguments():
    # Argument configuration
    parser = argparse.ArgumentParser()
    parser.add_argument('--query', type=str, required=True, help='query to search on NAVER')
    parser.add_argument('--begin', type=str, required=True, help='crawling begin point (%%Y.%%m.%%d format)')
    parser.add_argument('--end', type=str, required=True, help='crawling end point (%%Y.%%m.%%d format)')
    parser.add_argument('--save_as', type=str, default='test2.xlsx', help='excel save path')
    parser.add_argument('--sort', type=int, default=0, help='search result sorting: 0(relevant), 1(newest), 2(oldest)')
    parser.add_argument('--field', type=int, default=1, help='search field: 0(all), 1(title)')
    return parser.parse_args()

In [57]:
crawl('제주도관광', 'test.xlsx', '2015.01.01', '2015.12.31')


crawling... 제주도관광 (current_page / max_page 1/1)
making url https://search.naver.com/search.naver?&where=news&query=%EC%A0%9C%EC%A3%BC%EB%8F%84%EA%B4%80%EA%B4%91&sort=0&field=1&ds=2015.01.01&de=2015.12.31&nso=so:r,p:from20150101to20151231&start=1&refresh_start=0
making beautifulsoup object from html
extracting naver news urls from bsobj
['http://www.newsis.com/ar_detail/view.html?ar_id=NISX20151229_0010503813&cID=10813&pID=10800', 'http://www.nocutnews.co.kr/news/4524790', 'http://www.headlinejeju.co.kr/?mod=news&act=articleView&idxno=261210', 'http://www.newsis.com/ar_detail/view.html?ar_id=NISX20151228_0010501924&cID=10813&pID=10800', 'http://www.headlinejeju.co.kr/?mod=news&act=articleView&idxno=261324', 'http://news.jtbc.co.kr/html/088/NB11132088.html', 'http://www.ablenews.co.kr/News/NewsContent.aspx?CategoryCode=0039&NewsCode=003920151223090111629787', 'http://www.nocutnews.co.kr/news/4522373', 'http://news.donga.com/3/all/20151221/75487308/1', 'http://www.newsis.com/ar_detail/vi

/Users/jin-yulim/miniforge3/envs/tf25/lib/python3.8/site-packages/bs4/__init__.py:404: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


extracting naver news urls from bsobj
['http://www.newsis.com/ar_detail/view.html?ar_id=NISX20151217_0010483791&cID=10813&pID=10800', 'http://www.headlinejeju.co.kr/?mod=news&act=articleView&idxno=260832', 'http://www.headlinejeju.co.kr/?mod=news&act=articleView&idxno=261043', 'http://news1.kr/photos/view/?1694269', 'http://www.choicenews.co.kr/news/articleView.html?idxno=24257', 'http://www.jemin.com/news/articleView.html?idxno=378846', 'http://www.headlinejeju.co.kr/?mod=news&act=articleView&idxno=260330', 'http://www.jejusori.net/?mod=news&act=articleView&idxno=170723', 'http://www.headlinejeju.co.kr/?mod=news&act=articleView&idxno=259986', 'http://www.headlinejeju.co.kr/?mod=news&act=articleView&idxno=259923']
['제주도, 최우수 공영관광지에 절물휴양림 선정', '제주도 경제성장률 \'4.8%\'↑..."관광산업이 주도"', '제주도, 농어촌체험휴양마을 관광 활성화 워크숍 개최', "김종덕 문화체육관광부 장관 '제주도, 축하해요'", '[방송가 경제 진단] "제주도 영리병원, 중국 관광객 미용·성형 목적인가"', "제주도, 가을 관광주간 효과 '톡톡'", '제주도관광협회 관광인 송년 친선골프대회 개최', '제주도관광협회 신임 부회장 장성욱 위원장', "제주도 관광객 사상 첫 '1300만명' 돌파"

extracting naver news urls from bsobj
['http://www.headlinejeju.co.kr/news/articleView.html?idxno=257303', 'http://www.segye.com/content/html/2015/11/11/20151111000024.html?OutUrl=naver', 'http://www.kwnews.co.kr/view.asp?aid=215111000017&s=501', 'http://www.headlinejeju.co.kr/news/articleView.html?idxno=257011', 'http://www.jemin.com/news/articleView.html?idxno=375254', 'http://www.hankyung.com/news/app/newsview.php?aid=2015110839141', 'http://www.sisunnews.co.kr/news/articleView.html?idxno=29034', 'http://www.breaknews.com/sub_read.html?uid=407000§ion=sc2', 'http://app.yonhapnews.co.kr/YNA/Basic/SNS/r.aspx?c=AKR20151106163000056&did=1195m', 'http://www.breaknews.com/sub_read.html?uid=406809§ion=sc2']
['제주도관광협회, 일본 현지 관광로드홍보 추진', '제주신공항 서귀포 신산, 2025년 개항 목표..."제주도 관광 활성화"', '[동해]제주도 투어 관광열차 “출발”', '제주도, 인도네시아와 관광교류 협력 업무협약 체결', '제주도 중국 잠재 관광객 유치 나선다', '외국인 연 300만명 찾는 관광지 제주도의 민낯', '제주도 환상 자전거길 개통...234km의 아름다운 자전거 관광 만끽', '2018년, 제주도내 관광호텔 약 4,330실 이상 과잉공급 예상', '제주도 "기반시설 없는 자연녹지에 관광숙박

extracting naver news urls from bsobj
['http://news.jtbc.co.kr/html/900/NB11056900.html', 'http://www.mediapen.com/news/articleView.html?idxno=97612', 'http://www.gukjenews.com/news/articleView.html?idxno=347303', 'http://www.newsis.com/ar_detail/view.html?ar_id=NISX20151009_0010339509&cID=10813&pID=10800', 'http://www.gukjenews.com/news/articleView.html?idxno=345146', 'http://www.gukjenews.com/news/articleView.html?idxno=344633', 'http://www.mt.co.kr/view/mtview.php?type=1&no=2015100510380391430&outlink=1', 'http://www.jejudomin.co.kr/news/articleView.html?idxno=64932', 'http://ilyo.co.kr/?ac=article_view&entry_id=145108', 'http://www.newsis.com/ar_detail/view.html?ar_id=NISX20151002_0010326587&cID=10813&pID=10800']
['[뉴스브리핑] 한글날 제주도 하루 관광객 5만 5천명', '제주도 향한 요우커·국내관광객…일일관광객 역대 최고치', '제주도, 中 상해에서 크루즈관광객 유치활동 전개', '제주도 中 상하이서 크루즈관광객 유치활동', '[동정] 김영진 제주도관광협회 회장', '제주도-道관광협회, 온라인마켓 브랜드 네이밍 공모', '롯데관광, 제주도 드림타워 카지노 직접 추진', '관광산업 활성화 시켜 제주도내 습지 보전 한다고?', 'aT, 대학로, 제주도서 ‘국내외 관광객 대상 막걸리 홍보행사’ 

...

extracting naver news urls from bsobj
['http://mbn.mk.co.kr/pages/news/newsView.php?category=mbn00009&news_seq_no=2405785', 'https://n.news.naver.com/mnews/article/011/0002700489?sid=102', 'http://www.gvalley.co.kr/498791', 'http://www.imaeil.com/sub_news/sub_news_view.php?news_id=34635&yy=2015', 'http://www.etnews.com/20150619000159', 'http://star.mk.co.kr/new/view.php?mc=ST&no=586199&year=2015', 'http://news.donga.com/3/all/20150618/71946317/1', 'http://www.seoul.co.kr/news/newsView.php?id=20150619002008', 'http://beautyhankook.wowtv.co.kr/news/articleView.html?idxno=38058', 'http://www.fnnews.com/news/201506190103081847']
["제주 메르스 관광객, 감염 후 제주도 여행…'격리 대상서 제외된 이유는?'", '제주도 메르스, 원희룡 지사 "외국인 관광객 1/5로 급감해" 우려 표명', "제주도 메르스 발열 증상 호소한 관광객-도민 등 5명 '음성 판정'", '[채널] 이주 러시와 관광 개발로 몸살 앓는 제주도', "메르스 제주, 감염 확진자 제주도 관광 동선 공개 '엄청난 이동 경로'", '제주도 관광객 메르스 확진, 방문한 호텔·식당 공개', '‘메르스 확진 前 제주도 여행’ 관광객, 나흘 간 동선 자세히 살펴 보니?', '관광객에 ‘청정’ 제주도 뚫리나', '제주 관광객 메르스 확진, 여행 후 발열…제주도 메르스 영향은?', '제주 관광객 메르스 확진, 제주도 신라호텔

extracting naver news urls from bsobj
['http://news.donga.com/3/all/20150618/71946262/1', 'http://biz.chosun.com/site/data/html_dir/2015/06/18/2015061801731.html', 'http://www.ajunews.com/view/20150618165425240', 'http://news.kbs.co.kr/news/NewsView.do?SEARCH_NEWS_CODE=3097936&ref=A', 'http://www.tvreport.co.kr/?c=news&m=newsview&idx=730597', 'http://www.mbn.co.kr/pages/vod/programView.mbn?bcastSeqNo=1098901', 'http://sports.chosun.com/news/ntype.htm?id=201506190100232970016566&servicedate=20150618', 'http://www.sportsworldi.com/content/html/2015/06/18/20150618002952.html?OutUrl=naver', 'http://biz.heraldcorp.com/view.php?ud=20150618000977', 'http://mbn.mk.co.kr/pages/news/newsView.php?category=mbn00009&news_seq_no=2405535']
['‘메르스 확진 前 제주도 여행’ 관광객, 동선 살펴 보니? 호텔-승마장 등 이...', "제주도 메르스 환자 이동경로, '신라호텔'에서 머무르며 관광", '[제주도 관광객 메르스 확진]전문가“환자,감염원 역할 했을 가능성 낮아”', '특급 호텔·관광지 ‘발칵’…제주도 곳곳 ‘비상’', '신라호텔, 제주도 관광 메르스 환자 발생에 방역소독 실시', '[메르스] 40대 남성, 감염 후 제주도 관광', '제주도 메르스 관광객 확진, 제주신라호텔 "영업 잠정 중단…투숙객 퇴

extracting naver news urls from bsobj
['http://news.donga.com/3/all/20150618/71946730/1', 'http://sports.hankooki.com/lpage/sisa/201506/sp20150618145832137040.htm', 'http://news.donga.com/3/all/20150618/71946210/1', 'http://en.seoul.co.kr/news/newsView.php?id=20150618500355', 'http://star.mk.co.kr/new/view.php?mc=ST&no=583454&year=2015', 'http://biz.heraldcorp.com/view.php?ud=201506181928034245907_8', 'http://www.seoul.co.kr/news/newsView.php?id=20150618500343', 'https://n.news.naver.com/mnews/article/011/0002699899?sid=102', 'http://www.mediapen.com/news/articleView.html?idxno=80319', 'http://www.wowtv.co.kr/newscenter/news/view.asp?bcode=T30001000&artid=A201506180387']
['제주도 관광객 42세男 서울서 메르스 확진…관광객-면세점 직원 등 5명 메르스...', "신라호텔, 메르스 제주도 관광객 방문에 '비상' … '긴급 방역 실시'", '‘발열 증상’ 제주도 관광객-도민 등 5명, 메르스 검사 결과 모두 ‘음성 판...', '제주도 관광객 메르스 확진, 3박4일 동안 제주 관광.. 동선 보니', '메르스, 이제는 제주도까지…‘관광객’이 감염 ‘충격’', '제주도 메르스, 서귀포시 일대에서 관광..."제주신라호텔 영업 중단"', '[제주도 메르스 관광객] 진료소 나가며 “병 퍼뜨릴 것”…제주 보건당국 비상', "제주도 메르스 여파 6월

extracting naver news urls from bsobj
['http://www.wowtv.co.kr/newscenter/news/view.asp?bcode=T30001000&artid=A201506180473', 'http://star.mk.co.kr/new/view.php?mc=ST&no=583955&year=2015', 'https://ent.sbs.co.kr/news/article.do?article_id=E10006783090&plink=ORI&cooper=NAVER', 'http://www.hankyung.com/news/app/newsview.php?aid=2015061858817', 'http://www.g-enews.com/ko-kr/news/article/news_all/201506181514454560606_1/article.html', 'http://www.sportsseoul.com/?c=v&m=n&i=229341', 'http://sports.hankooki.com/lpage/sisa/201506/sp20150618134633137040.htm', 'http://star.mbn.co.kr/view.php?no=582746&year=2015&refer=portal', 'http://www.newsen.com/news_view.php?uid=201506181547502341', 'http://www.kookje.co.kr/news2011/asp/newsbody.asp?code=0300&key=20150618.99002181051']
['제주도 관광객 메르스 확진, 대구에선 무슨 일이? 이렇게까지…', '제주 관광객 메르스 확진…이젠 제주도 까지 확산되나', "제주도 메르스 관광객, 확진 판정…'정확한 동선 파악 어려워'", "제주 관광객 메르스 확진 판정, 3박4일 제주도 관광 즐겨…'초비상'", '제주도 메르스, 확진환자 지난 5일부터 8일까지 관광 "서귀포 중문, 남원읍 등...', '제주도 관광 141번 환자, 검사 도중 

extracting naver news urls from bsobj
['http://star.mt.co.kr/view/stview.php?no=2015061814031725108&type=1&outlink=1', 'http://news.kukinews.com/article/view.asp?arcid=0009560596&code=41121111&cp=nv', 'http://biz.chosun.com/site/data/html_dir/2015/06/16/2015061603492.html', 'http://www.joongboo.com/news/articleView.html?idxno=996184', 'http://mbn.mk.co.kr/pages/news/newsView.php?category=mbn00005&news_seq_no=2401542', 'http://www.headlinejeju.co.kr/news/articleView.html?idxno=244246', 'http://www.ebn.co.kr/news/view/767646', 'http://www.newsis.com/ar_detail/view.html?ar_id=NISX20150614_0013726753&cID=10813&pID=10800', 'http://www.newsis.com/pict_detail/view.html?pict_id=NISI20150612_0011045675', 'http://www.jejudomin.co.kr/news/articleView.html?idxno=61860']
['제주도 관광객, 메르스 확진..141번 환자 신라호텔등 이용', '제주도 관광 40대 남성, 메르스 확진 판정', '이효리 집 관광객 고충, 과거 블로그에 공개한 제주도 집 작업실', '이효리, 제주도 집에 넘쳐나는 관광객 "수십 차례 울리는 초인총으로 심한 스...', '관광객 증가로 제주도 숙박시설 8월까지 풀예약, 수익형 풀빌라 ‘라메종’ 화...', "제주도, 제30회 한국국제관광전 '최우수 마케팅상